In [147]:
import tweepy
import pandas as pd
import numpy as np

import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\naik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\naik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\naik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [148]:
# Add your Bearer token here.
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAAXMbQEAAAAAIVKCqVq%2FKfI67ramYlyV04omTBU%3Dg0MCZHkPmTNa7vBHCQIfuWvaYuO26AKojwwyLoiMNYXn97zYH1')

# Create Pandas dataframe with three columns
tweet_df = pd.DataFrame(columns=['text', 'language'])

# Query tweets with #news. Show only 100 tweets
query = '#news'
tweets = client.search_recent_tweets(query=query, tweet_fields=['lang'], max_results=10)

for tweet in tweets.data:
    text = tweet.text
    language = tweet.lang

    # Focus only to tweets in english
    if language == 'en':
        tmp = [text, language]
        tweet_df.loc[len(tweet_df)] = tmp

# Total amount of tweets in dataframe
print('Amount: ', len(tweet_df))

# Show first 5 lines of dataframe
tweet_df.head()

Amount:  7


,text,language
0,#Whatshappeninginvietnam ?\n\nFind out here wi...,en
1,RT @DemonSlayerUSA: #NEWS Get ready to embark ...,en
2,RT @DemonSlayerUSA: #NEWS Get ready to embark ...,en
3,RT @DemonSlayerUSA: #NEWS Get ready to embark ...,en
4,RT @DemonSlayerUSA: #NEWS Get ready to embark ...,en


In [185]:
truenews = pd.read_csv("../files/True.csv")
fakenews = pd.read_csv("../files/Fake.csv")

truenews['label'] = 1
fakenews['label'] = 0

news_df = pd.concat([truenews, fakenews])
news_df = news_df.sample(frac = 1).reset_index(drop=True)
news_df.head()

,title,text,subject,date,label
0,"Timeline: From TVs to microwaves, White House ...",WASHINGTON (Reuters) - The White House on Frid...,politicsNews,"March 17, 2017",1
1,Al Gore: Climate Change is “PRINCIPAL” Cause o...,21st Century Wire says Can you say: out of to...,US_News,"March 29, 2017",0
2,College Students Tell London Mayor To P*ss Of...,The Mayor of London s recent remarks about Bar...,News,"April 25, 2016",0
3,Facebook says Russian-linked accounts spent ju...,LONDON (Reuters) - Russian-based operatives pl...,worldnews,"December 13, 2017",1
4,"After Las Vegas massacre, Democrats urge gun l...",WASHINGTON (Reuters) - Sunday’s massacre in La...,politicsNews,"October 2, 2017",1


In [174]:
print("Does Twitter Dataframe contain null values: ", tweet_df.isnull().values.any())
print("Does News Dataframe contain null values: ", news_df.isnull().values.any(), "\n")

print("Twitter dataframe datatypes: \n", tweet_df.dtypes, "\n")
print("News dataframe datatypes: \n", news_df.dtypes)

Does Twitter Dataframe contain null values:  False
Does News Dataframe contain null values:  True 

Twitter dataframe datatypes: 
 text          object
language      object
clean_text    object
dtype: object 

News dataframe datatypes: 
 title          object
label           int64
clean_text    float64
dtype: object


In [151]:
def deEmojify(text):
    regex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regex_pattern.sub(r'', text)

def clean_text(tweet):
    # Lower case text
    tweet = tweet.lower()

    # Remove mentions
    tweet = re.sub("@[A-Za-z0-9]+","", tweet)

    # Remove links
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet)

    # Remove hastags
    tweet = re.sub("#[A-Za-z0-9_]+","", tweet)

    # Remove punctuations
    tweet = re.sub('[()!?]', ' ', tweet)
    tweet = re.sub('\[.*?\]',' ', tweet)

    # Remove emoticons
    tweet = deEmojify(tweet)

    # Filter non-alphanumeric characters (double check...)
    tweet = re.sub("[^a-z0-9]"," ", tweet)

    # Tokenize tweet (split...)
    tokens = word_tokenize(tweet)

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word, pos='v') for word in tokens]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]

    cleaned_tweet = ' '.join(tokens)

    return cleaned_tweet

test_text = "The @SpaceX Dragon Endeavour with four @Axiom_Space astronauts is holding 20 meters away from the station as the station crew works a video routing issue. https://nasa.gov/live"
print("Before: ", test_text)
print("After: ", clean_text(test_text))


Before:  The @SpaceX Dragon Endeavour with four @Axiom_Space astronauts is holding 20 meters away from the station as the station crew works a video routing issue. https://nasa.gov/live
After:  dragon endeavour four space astronauts hold 20 meter away station station crew work video rout issue


In [198]:
news_df['clean_text'] = np.nan

news_df['clean_text'] = [clean_text(x) for x in news_df['title']]

news_df.dropna(subset=['clean_text'], inplace=True)
news_df.drop_duplicates(subset=['text', 'clean_text'], keep=False, inplace=True)

news_df.head()

,title,text,subject,date,label,clean_text
0,"Timeline: From TVs to microwaves, White House ...",WASHINGTON (Reuters) - The White House on Frid...,politicsNews,"March 17, 2017",1,timeline tvs microwave white house try explain...
2,College Students Tell London Mayor To P*ss Of...,The Mayor of London s recent remarks about Bar...,News,"April 25, 2016",0,college students tell london mayor p ss racist...
3,Facebook says Russian-linked accounts spent ju...,LONDON (Reuters) - Russian-based operatives pl...,worldnews,"December 13, 2017",1,facebook say russian link account spend 97 cen...
4,"After Las Vegas massacre, Democrats urge gun l...",WASHINGTON (Reuters) - Sunday’s massacre in La...,politicsNews,"October 2, 2017",1,las vegas massacre democrats urge gun laws rep...
5,SECURITY ANALYST: Letting Muslim Refugees In I...,,politics,"Dec 23, 2016",0,security analyst let muslim refugees suicide pact


In [199]:
tweet_df['clean_text'] = np.nan

tweet_df['clean_text'] = [clean_text(x) for x in tweet_df['text']]

tweet_df.dropna(subset=['clean_text'], inplace=True)
tweet_df.drop_duplicates(subset=['text', 'clean_text'], keep=False, inplace=True)

tweet_df.head()

,text,language,clean_text
0,#Whatshappeninginvietnam ?\n\nFind out here wi...,en,find demand vietnam update every 30 minutes ev...
